## This is a debuging ipython notebook

In [ ]:
%load_ext autoreload
import argparse
import warnings
warnings.filterwarnings('ignore')

arg_parser = argparse.ArgumentParser()
        
    # # Add standard arguments
    # if not is_standalone:
    #     # If standalone, set in parser.
arg_parser.add_argument('--output_path', default='.',
                                help='[Default is current directory] Path to '
                                     'output directory.')


    # Arguments to generate the reference
arg_parser.add_argument('-r', '--reference', action='store_true',
                            help='Just generate the reference dataset for mapping')
arg_parser.add_argument('--min_species', type=int, default=30,
                            help='Min number of species in selected orthologous groups. \
                            If not selected it will be estimated such that around 1000 OGs are available.')
arg_parser.add_argument('--dna_reference', default='/Volumes/Untitled/eukaryotes.cdna.fa',
                            help='Reference fasta file that contains nucleotide sequences.')

    # Arguments to map the reads
arg_parser.add_argument('--ref_folder', default=None,
                            help='Folder containing reference files with sequences sorted by species.')
arg_parser.add_argument('--reads', default='/Users/daviddylus/Research/pore2tree/fritz_scripts/pipeline/Nano_reads/ERR1877969.fastq',
                            help='Reads to be mapped to reference.')
# arg_parser.add_argument('--reads', nargs='2',default=None, help='Reads to be mapped to reference.')


# Parse the arguments.
# reference = ['--ref_folder', '/Users/daviddylus/Research/pore2tree/pore2tree/tests/mapper/test1/db/']
argv = ['--output_path','/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26', 
        '--reads', '/Users/daviddylus/mnt/read2tree/mouse/reads/illumina_hiseq_encode/SRR5171076_02X_0_0.fq /Users/daviddylus/mnt/read2tree/mouse/reads/illumina_hiseq_encode/SRR5171076_02X_1_0.fq']
args = arg_parser.parse_args(argv)
print(args)

In [ ]:
import glob
output_path='/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/'
for files in glob.iglob(output_path+'/**/*consensus.fa', recursive=True):
    print(files)

In [ ]:
from Bio import AlignIO
import numpy as np
import re

for folder in glob.iglob(output_path+'/05_*', recursive=True):
    print(folder)
    all_coverages = []

    for file in glob.iglob(folder+'/*.phy'):
        align = AlignIO.read(file, "phylip-relaxed")
        for record in align:
            if 'SRR51' in record.id:
                seq = re.sub('-', '', str(record.seq))
                xx = seq.count("X")
                aa = len(seq)-xx
                all_coverages.append((aa/len(seq)))
    np_all_coverages = np.array(all_coverages)
    print(np.mean(np_all_coverages))
    print(np.std(np_all_coverages))


In [ ]:
from Bio import SeqIO
import numpy as np
import re

for folder in glob.iglob(output_path+'/04_*', recursive=True):
    print(folder)
    all_coverages = []

    for file in glob.iglob(folder+'/*.fa'):
        align = SeqIO.parse(file, "fasta")
        for record in align:
            if 'SRR51' in record.id:
#                 seq = re.sub('-', '', str(record.seq))
                xx = str(record.seq).count("X")
                aa = len(record.seq)-xx
                all_coverages.append((aa/len(record.seq)))
    np_all_coverages = np.array(all_coverages)
    print(np.mean(np_all_coverages))
    print(np.std(np_all_coverages))

In [ ]:
from Bio import SeqIO
from zoo.wrappers.aligners import Mafft
from tqdm import tqdm
import glob
import numpy as np
import os
import re
output_path='/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/'
mapping='/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/04_ogs_map_SRR5171076_09X_0_0/'
real='/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/m_bg25/01_ref_ogs_aa/'
all_diff = {}
for folder in glob.iglob(output_path+'/04_*', recursive=True):
    out_dict = {}
    print(folder)
    for file in tqdm(glob.iglob(folder+"/*.fa"), desc='Loading OGs ', unit=' ogs'):
        to_check = []
        map_rec = SeqIO.parse(file, "fasta")
        for record in map_rec:
            if 'SRR51' in record.id:
                to_check.append(record)
        if os.path.exists(real+os.path.basename(file)):
            ori_rec = SeqIO.parse(real+os.path.basename(file), "fasta")
            for record in ori_rec:
                if 'MOUSE' in record.id:
                    to_check.append(record)
                    out_dict[os.path.basename(file)] = to_check
    align_dict = {}
    for key, value in tqdm(out_dict.items(), desc='Aligning OGs ', unit=' alignments'):
        mafft_wrapper = Mafft(value, datatype="PROTEIN")
        mafft_wrapper.options.options['--localpair'].set_value(True)
        mafft_wrapper.options.options['--maxiterate'].set_value(1000)
        alignment = mafft_wrapper()
        align_dict[key] = alignment
    
    differences = []
    differences_dict = {}
    for key,alignment in align_dict.items():
        differences.append(get_align_diff(alignment))
        differences_dict[key] = get_align_diff(alignment)
    all_diff[folder] = differences_dict
    diff_np = np.array(differences)
    print(diff_np.mean())
    print(diff_np.std())

In [ ]:
with open('csvfile.csv','w') as file:
    for key,value in all_diff.items():
        for key2,value2 in value.items():
            #print(key.split('_map_')[-1]+','+key2+','+'{}'.format(value2))
            file.write(key.split('_map_')[-1]+','+key2+','+'{}'.format(value2)+'\n')
    

In [ ]:
def get_align_diff(alignment):
    diff = 0
    for i,value in enumerate(alignment[0].seq):
        if value is not 'X' and value is not '-':
            if alignment[0][i] is not alignment[1][i]:
                diff += 1
    return diff

In [ ]:
import pyopa
defaults = pyopa.load_default_environments()
envs = defaults['environments']
env = envs[515]
align_dict_pyopa = {}
for key, value in out_dict.items():
    s1 = pyopa.Sequence(str(value[0].seq))
    s2 = pyopa.Sequence(str(value[1].seq))
    align_dict_pyopa[key] = pyopa.align_double(s1, s2, env)

In [ ]:
differences_dict

In [ ]:
dir(pyopa)

In [5]:
import os
from tqdm import tqdm
import glob
from Bio import SeqIO, Seq, SeqRecord
from Bio.Alphabet import SingleLetterAlphabet
from Bio.SeqIO.FastaIO import FastaWriter
mapping='/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/wm_bg26/04_ogs_map_SRR5171076_01X_0_0/'
real='/Users/daviddylus/mnt/read2tree/mouse/read2tree_out/SRR5171076/m_bg25/01_ref_ogs_aa/'
to_check = []
for file in tqdm(glob.iglob(mapping+"*.fa"), desc='Loading OGs ', unit=' ogs'):
        map_rec = SeqIO.parse(file, "fasta")
        for record in map_rec:
            if 'SRR51' in record.id:
                to_check.append(record)
#         if os.path.exists(real+os.path.basename(file)):
#             ori_rec = SeqIO.parse(real+os.path.basename(file), "fasta")
#             for record in ori_rec:
#                 if 'MOUSE' in record.id:
#                     to_check.append(record)

handle = open('/Users/daviddylus/Desktop/01x.fa', "w")
writer = FastaWriter(handle, wrap=None)
writer.write_file(to_check)

Loading OGs : 495 ogs [00:04, 105.81 ogs/s]


495